#### Importing Packages

In [1]:
import pylab
import h5py
import math
import array
from numpy import *
import numpy as np
from pycbc.types import TimeSeries, FrequencySeries
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.waveform.waveform_modes import get_td_waveform_modes
from pycbc import types, fft, waveform
import lal
from scipy import interpolate
from scipy.interpolate import interp1d
from lal import MSUN_SI, MTSUN_SI, G_SI, PC_SI, C_SI, PI
from pycbc.filter import match
from pycbc.psd import aLIGOZeroDetHighPower
from tqdm import tqdm#


import matplotlib as mpl

 
from matplotlib import gridspec
from matplotlib import ticker

import matplotlib.pyplot as plt
import pandas as pd
import csv

#### Importing Eccentricity - Frequency data for a particular simulation

In [14]:
df = pd.read_csv('Ecc vs Freq_1371.csv')

In [15]:
F = []
E = []

for i in df['Eccentricity']:
    E.append(i)
for j in df['Frequency']:
    F.append(j)
    
print(len(E),len(F))

4161 4161


#### This block is needed only when we fix either frequency or eccentricity

In [16]:
F1 = []
E1 = []
for i in range(len(F)):
    if abs(E[i]-0.12)<1e-4:#abs(F[i]-20)<1e-2:
        F1.append(F[i])
        E1.append(E[i])
print(len(E1),len(F1))
print(min(E1))

4 4
0.1199219300299337


#### Moore et.al. definition of eccentricity

In [17]:
# Eq. (4.17a, 4.17b), Pg. 18, Moore et al (2016)

def epsilon(xi, eta):
    return(( 1 + ( ( -2833/2016 + 197/72 * eta ) * ( xi )**( 2/3 ) + 
                  ( -377/144 * np.pi * xi + ( ( 77006005/24385536 + ( -1143767/145152 * eta + 
 	 43807/10368 * ( eta )**( 2 ) ) ) * ( xi )**( 4/3 ) + ( np.pi * ( 9901567/1451520 + 
 	 -202589/362880 * eta ) * ( xi )**( 5/3 ) + ( xi )**( 2 ) * ( -33320661414619/386266890240 + 
 	 ( 3317/252 * EulerGamma + ( 180721/41472 * ( np.pi )**( 2 ) + ( ( 161339510737/8778792960 + 
 	 3977/2304 * ( np.pi )**( 2 ) ) * eta + ( -359037739/20901888 * ( eta )**( 2 ) + 
      ( 10647791/2239488 * ( eta )**( 3 ) + ( -87419/3780 * np.log( 2 ) + 
 	 ( 26001/1120 * np.log( 3 ) + 3317/504 * np.log( 16 * ( xi )**( 2/3 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ))

#### Analytical fits expressions

In [18]:
# "Hinder+ modified all 20 simulations SEOBNRv4 model, full frequency range --Feb 16
g=open('tshift_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
A=[] 
for x in lines: 
    A.append(float(x.split()[1])) 
g.close()

def tshift_Hinsp(q,e,l): 
    return A[0] + A[1]*q + A[2]*q**2 + A[3]*e + A[4]*e**2 + A[5]*e**3 + A[6]*e*q + A[7]*(e**2)*q + A[8]*e*(q**2) + A[9]*(e**2)*(q**2) + A[10]*(e**3)*q + A[11]*e*q*math.cos(l + A[12]) + A[13]*(e**2)*(q**2)*math.cos(e*l + A[14]) + A[15]*(e**3)*q*math.cos(l + A[16]) + A[17]*e*(q**2)*math.cos(l + A[18])

g=open('tamp_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
B=[] 
for x in lines: 
    B.append(float(x.split()[1])) 
g.close()

def tamp_Hinsp(eta,e,l): 
    return B[0] + B[1]*eta + B[2]*eta**2 + B[3]*e + B[4]*e**2 + B[5]*e**3 + B[6]*e*eta + B[7]*(e**2)*eta + B[8]*e*(eta**2) + B[9]*(e**2)*(eta**2) + B[10]*(e**3)*eta + B[11]*e*eta*math.cos(l + B[12]) + B[13]*(e**2)*(eta**2)*math.cos(l + B[14]) + B[15]*(e**3)*eta*math.cos(e*l + B[16])

g=open('tfreq_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
C=[] 
for x in lines: 
    C.append(float(x.split()[1])) 
g.close()

def tfreq_Hinsp(eta,e,l): 
    return C[0] + C[1]*eta + C[2]*eta**2 + C[3]*e + C[4]*e**2 + C[5]*e**3 + C[6]*e*eta + C[7]*(e**2)*eta + C[8]*e*(eta**2) + C[9]*(e**2)*(eta**2) + C[10]*(e**3)*eta + C[11]*e*eta*math.cos(l + C[12]) + C[13]*(e**2)*(eta**2)*math.cos(l + C[14]) + C[15]*(e**3)*eta*math.cos(e*l + C[16])+ C[17]*(e**3)*(eta**2)*math.cos(e*l + C[18])

#### Spherical Harmonics 

In [19]:
def sph_harmonics(inc,ell):
    L=ell
    #inc = 10
    theta = inc
    for l in range(L,L+1):

        for m in range(-l,l+1):
            dlm = 0;
            k1 = max([0, m-2]);
            k2 = min([l+m, l-2]);

            #if(m==l or m==l-1):
            for k in range(k1,k2+1):
                A = []; B = []; cosTerm = []; sinTerm = []; dlmTmp = [];

                A = (-1)**k*math.sqrt(math.factorial(l+m)*math.factorial(l-m)*math.factorial(l+2)*math.factorial(l-2));
                B = math.factorial(k)*math.factorial(k-m+2)*math.factorial(l+m-k)*math.factorial(l-k-2);

                cosTerm = pow(math.cos(theta/2), 2*l+m-2*k-2);
                sinTerm = pow(math.sin(theta/2), 2*k-m+2);

                dlmTmp = (A/B)*cosTerm*sinTerm;
                dlm = dlm+dlmTmp

            Ylm = math.sqrt((2*l+1)/(4*math.pi))*dlm
            #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
            if m==ell:
                #globals()['sph' + str(l) + str(m)] = Ylm
                #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
                sphlm = Ylm
            elif m==-ell:
                #globals()['sph' + str(l) + '_' + str(abs(m))] = Ylm
                #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
                sphl_m = Ylm
            else:
                continue
    return sphlm, sphl_m

#### Frequency convertion factors

In [20]:
def xi(x):
    return x**(3/2)

def xconv(f,M):
    return (PI*M*MTSUN_SI*f)**(2/3)  #22 mode conversion

def fconv(x,M):
    return x**(3/2)/(PI*M*MTSUN_SI)  #22 mode conversion


#### Function that can call each $(l, m)$ model for match calculations

In [21]:
def eccmodel(Mass,q0,e0,l0,fmin,inclination=0,d=1,delta_t=1./4096,modes=[[2,2]]):
    
    #delta_t=0.00015208911520102518
    #delta_t = 1/2**20
    ell = []
    numrows = len(modes)
    numcols = len(modes[0])
    for i in range(0,numrows):
        l = modes[i][0]
        m = modes[i][1]
        ell.append(l)
    angle = inclination
    waveform = {}
    count = 0
    el = 2
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = model_22(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l2_m2'] = mode_data
        count = count + 1

    el = 3
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL33(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l3_m3'] = mode_data
        count = count + 1
        
    el = 4
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL44(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l4_m4'] = mode_data
        count = count + 1
        
    el = 5
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL55(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l5_m5'] = mode_data
        count = count + 1
        
    len_max_mode = '0'
    len_max = 0
    for mode in waveform.keys():
        if len(waveform[mode]['t'])>len_max:
            len_max_mode = mode
            len_max = len(waveform[mode]['t'])
            
    for mode in waveform.keys():
        if mode != len_max_mode:
            waveform[mode]['hp'].resize(len_max)
            waveform[mode]['hc'].resize(len_max)
            
    hp=0
    hc=0
    time=waveform[len_max_mode]['t']
    for mode in waveform.keys():
        hp = hp + waveform[mode]['hp']
        hc = hc + waveform[mode]['hc']
        
    hplus = TimeSeries(hp,delta_t,epoch=time[0])
    hcross = TimeSeries(hc,delta_t,epoch=time[0])
    
    return hplus, hcross

#### Function that generates PN and NR waveforms given the parameter values

In [22]:
def PN_NR(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    M=m
    q=q0
    M1 = q*M/(1+q)
    M2 = M/(1+q)
    eta = q/(1+q)**2

    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    mode2polfac=4*(5/(64*np.pi))**(1/2)


    fref = 0.075**(3/2) /MTSUN_SI/ PI/M
    fmin = fmin

    #PN
    hp, hc = get_td_waveform(approximant='EccentricTD', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin, eccentricity=e0)
        
        
    #NR
    simulation = '/home/pratul/Downloads/Project/NR_data/1371_rhOverM_Asymptotic_GeometricUnits_CoM.h5'
    sims = simulation.split('_')[0]
    lp = 2 #modes
    mp = 2
    tref = 320
    mode = 'l'+str(lp)+'_m'+str(mp)
    with h5py.File(simulation, 'r') as hdf:
        temp=hdf['OutermostExtraction.dir']['Y_'+mode+'.dat']
        test=np.array(temp)

    x1=test[:,0]
    y1=test[:,1]    
    z1=test[:,2]


    plotband=np.where(x1>=tref)
    x1=x1[plotband]
    x1=x1-x1[np.argmax(abs(y1))]
    y1=y1[plotband]    
    z1=z1[plotband]
    x1 = x1-x1[np.argmax(y1)] #setting t_merger = 0
    
    
    NRAmp = abs(y1-1j*z1)

    tot=y1-1j*z1
    amp=abs(tot)
    ph=np.angle(tot)
    phase_NR=np.unwrap(ph)
    w=np.absolute(diff(phase_NR)/diff(x1))
    phi0=phase_NR[0]
                
   # interpolation EccentricTD
    hp_intrp = interp1d(hp.sample_times/(M*MTSUN_SI), hp/(G_SI*M_SI/D_SI/C_SI/C_SI * mode2polfac), kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(hc.sample_times/(M*MTSUN_SI), hc/(G_SI*M_SI/D_SI/C_SI/C_SI * mode2polfac), kind='cubic',fill_value='extrapolate')
    time_PN = np.arange(-2000, -1000, delta_t) #x1[0]
    hpVec_PN = hp_intrp(time_PN)
    hcVec_PN = hc_intrp(time_PN)
    #h22Ecc = hpVec_PN + 1j*hcVec_PN
    
    # interpolation NR
    hpVec_NR_intrp = interp1d(x1, y1, kind='cubic', fill_value = 'extrapolate')
    hcVec_NR_intrp = interp1d(x1, z1, kind='cubic', fill_value = 'extrapolate')
    time_NR = np.arange(-2000,-1000,delta_t) #x1[0]
    hpVec_NR = hpVec_NR_intrp(time_NR)
    hcVec_NR = hcVec_NR_intrp(time_NR)
            
    #plt.figure(figsize=(12,4))
    #plt.plot(time_PN, hpVec_PN,color='cyan',label='PN',linewidth=1)
    #plt.plot(time_NR, hpVec_NR,label='NR',color='darkblue')
    #plt.legend()
    #plt.xlim(-4100,100)
    
    return time_PN, hpVec_PN, hcVec_PN, time_NR, hpVec_NR, hcVec_NR
    
    
    
        

#### Mismatch Calculation between PN and NR by choosing a particular window

In [27]:
M = 30
q0 = 3
l0 = 0.665
d0 = 1
inc = 0
delta_t = 1./4096
D_SI = (10**(6))*PC_SI
mode2polfac=4*(5/(64*np.pi))**(1/2)
Mismatch = []

for k in tqdm(range(len(F))):
    time_PN, hpVec_PN, hcVec_PN, time_NR, hpVec_NR, hcVec_NR= PN_NR(30,3,E[k],0.665,F[k],0,1,1./4096)
    
        
    Hp = TimeSeries(hpVec_PN, delta_t, epoch=0)
    Hc = TimeSeries(hcVec_PN, delta_t, epoch=0)
    HpVec_NR = TimeSeries(hpVec_NR, delta_t)
    HcVec_NR = TimeSeries(hcVec_NR, delta_t)
    
    
    HP = HpVec_NR*(G_SI*M*MSUN_SI/D_SI/C_SI/C_SI * mode2polfac)
    HC = HcVec_NR*(G_SI*M*MSUN_SI/D_SI/C_SI/C_SI * mode2polfac)
    PHASE = (np.unwrap(np.angle(HP-1j*HC)*2)/2)
    delta_t =  np.abs(np.mean((np.diff(time_PN*30*MTSUN_SI))))
    OMEGA22 = (1/delta_t)*(np.gradient(PHASE))
    low_freq_cutoff = (OMEGA22/(2*PI))[0]
    
    
    # mismatch
    tlen = max(len(Hp),len(HpVec_NR))
    Hp.resize(tlen)
    HpVec_NR.resize(tlen)
    delta_f = 1./(Hp.duration)
    flen = tlen//2+1
    f_low = 15
    psd = aLIGOZeroDetHighPower(flen, delta_f, f_low)
    m, i =match(HpVec_NR,Hp,psd=psd,low_frequency_cutoff=low_freq_cutoff)
    mismatch = 1-m
    #print(1-mismatch)
    Mismatch.append(mismatch)

  0%|                                        | 1/4161 [00:02<2:20:53,  2.03s/it]

0.9999999784933241


  0%|                                        | 2/4161 [00:04<2:20:53,  2.03s/it]

0.9944481150156688


  0%|                                        | 3/4161 [00:06<2:22:22,  2.05s/it]

0.9944481150156688


  0%|                                        | 4/4161 [00:08<2:23:26,  2.07s/it]

0.9944481150156688


  0%|                                        | 4/4161 [00:09<2:36:06,  2.25s/it]


KeyboardInterrupt: 

#### Finding out the eccentricity-frequency pair corresponding to lowest mismatch

In [18]:
MMM = np.delete(Mismatch,0)
ind=np.where(MMM==np.min(MMM))[0][0]
print('e, f = ',E[ind],',',F[ind],ind,1-MMM[ind])

e, f =  0.173290800750527 , 16.526461447424033 69 0.9801490969127337
